In [252]:
from functools import cached_property, cache, reduce

class Droplet(object):

    def __init__(self, file) -> None:
        self._file = file
        self.load()

    def load(self):
        with open(self._file,'r') as file:
            voxels = [tuple([int(x) for x in line.split(',')]) for line in file.read().splitlines()]

        self.voxels = set(voxels)

    @cache
    def neighbour(self,voxel,location: tuple[int,int,int]):
        return tuple([x+y for x,y in zip(voxel,location)])
        
    def neighbour_exists(self,voxel,location,air=False):


        nb = self.neighbour(voxel,location)
        if self.oob(nb):
            return False
        elif nb in self.voxels and not air:
            return True
        elif nb not in self.voxels and air:
            return True
        else:
            return False

    def neighbours(self,voxel,air=False):
        options = [
            (1,0,0),
            (-1,0,0),
            (0,1,0),
            (0,-1,0),
            (0,0,1),
            (0,0,-1)
        ]

        if self.oob(voxel):
            raise ValueError(f'voxel {voxel} is out of bounds')
        else:
            return [self.neighbour(voxel,n) for n in options if self.neighbour_exists(voxel,n,air)]


    def oob(self,voxel):
        return any([x-y < 0 for x,y in zip(voxel,self.extent[0])]) or \
            any([x-y >= 0 for x,y in zip(voxel,self.extent[1])])


    @cached_property
    def extent(self, margin=2):       
        return (
            tuple([min(x)-margin for x in list(zip(*self.voxels))]),
            tuple([max(x)+margin for x in list(zip(*self.voxels))])
        )

    def surface_area(self, voxels = None, air=False):
        
        if voxels is None:
            voxels = self.voxels

        sides = 0

        for voxel in voxels:
            sides += (6 - len(self.neighbours(voxel, air=air)))

        return sides

    def flood_fill(self, air=True):
        if air:
            stack = [self.extent[0]]
        else:
            stack = [list(self.voxels[0])[0]]
        filled_voxels = []
        while stack:
            voxel = stack.pop()
            if voxel in filled_voxels:
                continue
            else:
                filled_voxels.append(voxel)
                stack.extend(self.neighbours(voxel,air))

        return set(filled_voxels)

    @property
    def _all(self):
        rngs = [range(x,y) for x,y in zip(*self.extent)]
        return set([(x,y,z) for z in rngs[2] for y in rngs[1] for x in rngs[0]])

    @property
    def air_pockets(self):
        return self._all - self.flood_fill() - self.voxels  

    def outer_surface(self):
        return self.surface_area(self.voxels) - self.surface_area(self.air_pockets, air=True)

            

In [254]:
droplet = Droplet('./assets/input_day_18.txt')
print(f"the total surface area of the droplet is: {droplet.surface_area()}")
print(f"the outer surface area of the droplet is: {droplet.outer_surface()}")


the total surface area of the droplet is: 4400
the outer surface area of the droplet is: 2522
